# Targets for Abi

She's on at Magellan June 3 and 22, 2022, and needs second-half-night targets.

| field | ra | dec|
|------------|--------|----------|
|COSMOS-1    | 150.0  |  3.1     |
|COSMOS-2    | 149.22 | 1.75     |
|COSMOS-3    | 150.78 | 1.75     |
|DECaPS-East | 270.89 | -29.53   |
|ELAIS-E1    | 7.8746 | -43.0096 |
|ELAIS-E2    | 9.5    | -43.9980 |
|DECaPS-West | 116.32 | -26.25   |

## June 3

There are no new ELAIS targets yet.

COSMOS has set, as has DECaPS-West.

So we're looking for DECaPS-East targets.

In [1]:
import psycopg2
import psycopg2.extras
import getpass
# from IPython.display import display
# import ipywidgets

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# import pandas as pd

from copy import deepcopy
import time
from astropy.time import Time

font = {'size' : 15}
mpl.rc('font', **font)

In [2]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")

DB User:  mgraham
DB Password:  ············


In [3]:
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

db.autocommit = True # Makes the computer nicer about SQL syntax errors

cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

In [ ]:
### If you want to print table schema
# tables = ['exposures','subtractions','objects','objectrbs','candidates']
# tables = ['exposures','subtractions','objects','candidates']
# for table in tables:
#     query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name=%s"
#     cursor.execute( query, ( table, ))
#     print( f"\nTABLE: {table}\n===========================" )
#     for row in cursor:
#         print( f"{row['column_name']:24s}  :  {row['data_type']:s}" )

### Look for recent candidates in DECaPS-East

In [4]:
t1 = time.time()

query = ("SELECT id, ra, dec, created_at, modified FROM candidates "
         "WHERE ra > 268 AND ra < 273 AND dec < -27 AND dec > -32")

cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()

t2 = time.time()
print(t2-t1)
del t1,t2

cand_id       = np.asarray( results[0] )
cand_ra       = np.asarray( results[1], dtype='float' )
cand_dec      = np.asarray( results[2], dtype='float' )
cand_created  = np.asarray( results[3], dtype='str' )
cand_modified = np.asarray( results[4], dtype='str' )
del query, results

print(len(cand_id))

22.438858032226562
1348524


In [5]:
print('Dates of DECaPS-East candidates, first and last created.')
sx = np.argsort( cand_created )
print(cand_created[sx[0]])
print(cand_created[sx[-1]])
print(' ')
print('Dates of DECaPS-East candidates, first and last modified.')
sx = np.argsort( cand_modified )
print(cand_modified[sx[0]])
print(cand_modified[sx[-1]])
print(' ')
print('There does not seem to be any DECaPS-East candidates from 2022A, started April 2022')

Dates of DECaPS-East candidates, first and last created.
2021-06-03 15:35:17.368927
2022-03-30 10:30:54.504370
 
Dates of DECaPS-East candidates, first and last modified.
2021-06-03 15:35:17.368927
2022-03-30 10:30:54.504370
 
There does not seem to be any DECaPS-East candidates from 2022A, started April 2022


### Look for recent objects in DECaPS-East

In [6]:
t1 = time.time()

query = ("SELECT o.id, o.ra, o.dec, o.candidate_id, o.subtraction_id, o.mag, o.magerr, o.ignore, "
         "rbs.rb, sub.id, sub.exposure_id, exp.mjd "
         "FROM objects AS o "
         "JOIN objectrbs AS rbs ON o.id=rbs.object_id AND rbs.rbtype_id=1 "
         "JOIN subtractions AS sub ON sub.id=o.subtraction_id "
         "JOIN exposures AS exp ON exp.id=sub.exposure_id "
         "WHERE o.ra > 268 AND o.ra < 273 AND o.dec < -27 AND o.dec > -32")
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()

t2 = time.time()
print(t2-t1)
del t1,t2

obj_id     = np.asarray( results[0], dtype='int' )
obj_ra     = np.asarray( results[1], dtype='float' )
obj_dec    = np.asarray( results[2], dtype='float' )
obj_candid = np.asarray( results[3] )
obj_subid  = np.asarray( results[4], dtype='int' )
obj_mag    = np.asarray( results[5], dtype='float' )
obj_magerr = np.asarray( results[6], dtype='float' )
obj_ignore = np.asarray( results[7] )
obj_rb     = np.asarray( results[8], dtype='float' )
obj_subid  = np.asarray( results[9], dtype='int' )
obj_expid  = np.asarray( results[10], dtype='int' )
obj_mjd    = np.asarray( results[11], dtype='float' )

del query, results

print(len(obj_id))

105.28891634941101
3732716


In [8]:
sx = np.argsort(obj_mjd)

temp1 = Time(obj_mjd[sx[0]], format='mjd')
temp2 = Time(obj_mjd[sx[-1]], format='mjd')

print('First object detected in DECaPS-East on: ',temp1.iso)
print('Most recent object detected in DECaPS-East on: ',temp2.iso)

First object detected in DECaPS-East on:  2021-03-22 09:13:33.030
Most recent object detected in DECaPS-East on:  2021-10-23 00:32:37.663
